In [ ]:
import re
import nltk
import random
import string
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from nltk import word_tokenize
from collections import Counter
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

nltk.download('stopwords')
nltk.download('punkt')
plt.figure(figsize=(10, 10))


In [ ]:
# Download and unzip translation dataset
!wget https://www.manythings.org/anki/spa-eng.zip
!unzip spa-eng.zip
text_file = 'spa.txt'

In [ ]:
# Parse data, place start/end tokens on target sequence and append source/target text pairs 
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]

text_pairs = []
for line in lines:
    english, spanish, *rest = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [ ]:
# Display a few line pairs
for x in range(5):
    print(random.choice(text_pairs))

In [ ]:
# Split data into training, validation and testing datasets
train_ratio = 0.70
val_ratio = 0.15
test_ratio = 1 - train_ratio - val_ratio 

num_train = int(train_ratio * len(text_pairs))
num_val = int(val_ratio * len(text_pairs))

random.shuffle(text_pairs)
train = text_pairs[:num_train]
val = text_pairs[num_train: num_train + num_val]
test = text_pairs[num_train + num_val:]

print("Training:   " + str(len(train)))
print("Validation: " + str(len(val)))
print("Testing:    " + str(len(test)))
print("Total:     " + str(len(text_pairs)))

In [ ]:
#Hyperparamters 
vocab_size = 15000
sequence_length = 20
batch_size = 64

embed_dim = 256
latent_dim = 2048
num_heads = 8
epochs = 40 

# custom standardization for target sequence, remove additional punctuations  
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
  
def standardize(string):
    lowercase = tf.strings.lower(string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")

# Vectorize source and target sequences to create two vocabularies 
english_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)

spanish_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=standardize,
)

train_english_texts = [pair[0] for pair in train]
train_spanish_texts = [pair[1] for pair in train]
english_vectorization.adapt(train_english_texts)
spanish_vectorization.adapt(train_spanish_texts)

In [ ]:
# Word frequency distribution for english 
data = train_english_texts
ngram_vectorizer = CountVectorizer(analyzer='word', stop_words="english", tokenizer=word_tokenize, ngram_range=(1, 1), min_df=1)
X = ngram_vectorizer.fit_transform(data)

english_vocab = list(ngram_vectorizer.get_feature_names())
english_counts = X.sum(axis=0).A1 # efficent numpy column-wise sum of X matrix
freq_dist_eng = Counter(dict(zip(english_vocab, english_counts)))


In [ ]:
fd = nltk.FreqDist(freq_dist_eng)
fd.plot(25)
print(freq_dist_eng.most_common(25))

In [ ]:
# Word frequency distribution for spanish 
spanish_text = []
spanish_text = filter(lambda x: x not in stopwords.words('spanish'), train_spanish_texts)

data2 = spanish_text
ngram_vectorizer = CountVectorizer(analyzer='word', tokenizer=word_tokenize, ngram_range=(1, 1), min_df=1)
Y = ngram_vectorizer.fit_transform(data2)

spanish_vocab = list(ngram_vectorizer.get_feature_names())
spanish_counts = Y.sum(axis=0).A1
freq_dist_spa = Counter(dict(zip(spanish_vocab, spanish_counts)))

In [ ]:
fd2 = nltk.FreqDist(freq_dist_spa)
fd2.plot(25)
print(freq_dist_spa.most_common(25))

In [ ]:
# Format training and validation sets as following:
# inputs: entire vectorized source sequence and generated target sequence so far
# output: target sequence offsetted by one -- model will predict the new word 

def format_dataset(english, spanish):
    english = english_vectorization(english)
    spanish = spanish_vectorization(spanish)
    return ({"encoder_inputs": english, "decoder_inputs": spanish[:, :-1],}, spanish[:, 1:])

def make_dataset(pairs):
    english_texts, spanish_texts = zip(*pairs)
    english_texts = list(english_texts)
    spanish_texts = list(spanish_texts)
    dataset = tf.data.Dataset.from_tensor_slices((english_texts, spanish_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train)
val_ds = make_dataset(val)

In [ ]:
# Verify input and output shapes match 
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

In [ ]:
# Sequence-to-Sequence Transformer Architecture  

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True
        
# override get_config method for custom layer classes
# This will allow proper saving and loading of models         
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'embed_dim': self.embed_dim,
            'dense_dim': self.dense_dim,
            'num_heads': self.num_heads,
        })
        return config

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'sequence_length': self.sequence_length,
            'vocab_size': self.vocab_size,
            'embed_dim': self.embed_dim,
        })
        return config        
        
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True
        
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'embed_dim': self.embed_dim,
            'latent_dim': self.latent_dim,
            'num_heads': self.num_heads,
        })
        return config

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [ ]:
# Construct end-to-end Transformer using encoder and decoder components 

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

transformer.summary()

In [ ]:
# Configure and train model 
transformer.compile(optimizer="rmsprop",
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])

history = transformer.fit(train_ds, epochs=1, validation_data=val_ds)

In [ ]:
# Plot the training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# Save/upload trained model 

#transformer.save('/content/drive/MyDrive/transformer_X_Epochs.h5')

'''
model_path = '/content/drive/MyDrive/transformer_X_Epochs.h5'
transformer = tf.keras.models.load_model(model_path, custom_objects={
    'TransformerEncoder': TransformerEncoder,
    'PositionalEmbedding': PositionalEmbedding,
    'TransformerDecoder': TransformerDecoder }
    )
'''

In [ ]:
# Check model performance on testing dataset
# Test samples will needed to be properly formatted 

spanish_vocab = spanish_vectorization.get_vocabulary()
spanish_index_lookup = dict(zip(range(len(spanish_vocab)), spanish_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = english_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spanish_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spanish_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence

test_english_texts = [pair[0] for pair in test]
for x in range(10):
    input_sentence = random.choice(test_english_texts)
    translated = decode_sequence(input_sentence)
    print(input_sentence + " " + translated)

In [ ]:
# Test user-generated sentences
user_input = input("Please enter an English sentence to be translated into Spanish: ")
translated_input = decode_sequence(user_input)
print(user_input + " " + translated_input)